In [ ]:
#################################This one#########################################
import re
from datetime import datetime
import pandas as pd
import zipfile
import numpy as np
import spacy

nlp = spacy.load("en_core_web_sm")

names = []
birthyears = []
nobirthyear = []
adults = []
minors = []
notborn = []
generations = ['issei', 'nisei', 'sansei', 'yonsei', 'kibei', 'ni-ten-gosei', 'nisei-sansei']
issei = []
nisei = []
sansei = []
yonsei = []
kibei = []
ni_ten_gosei = []
nisei_sansei = []
nogeneration = []
camp_locations = ["Gila River", "Granada (Amache)", "Amache", "Heart Mountain", "Jerome", "Manzanar", "Minidoka", "Poston (Colorado River)","Poston concentration camp", "Rohwer", "Topaz (Central Utah)", "Topaz concentration camp", "Tule Lake", "Fresno", "Marysville", "Mayer", "Merced", "Owens Valley", "Parker Dam",
    "Pinedale", "Pomona", "Portland", "Puyallup (Camp Harmony)","Puyallup Assembly Center", "Sacramento (Walerga)", "Salinas", "Santa Anita", "Stockton", "Tanforan", "Tulare", "Turlock", "East Boston", "Ellis Island", "San Francisco", "San Pedro", "Seattle", "Sharp Park",
    "Tuna Canyon", "Crystal City", "Fort Lincoln (Bismarck)", "Bismarck", "Fort Missoula", "Fort Stanton", "Kenedy", "Kooskia", "Old Raton Ranch", "Santa Fe", "Seagoville", "Camp Blanding", "Camp Forrest", "Camp Livingston", "Camp McCoy", "Florence", "Fort Bliss",
    "Fort Howard", "Fort Lewis", "Fort McDowell", "Fort Meade", "Fort Richardson", "Fort Sam Houston", "Fort Sill", "Griffith Park", "Honouliuli", "Lordsburg", "Sand Island", "Stringtown", "Leupp", "Moab", "Antelope Springs", "Cow Creek", "Nyssa, Oregon (detention facility)",
    "Tulelake", "Catalina Federal Honor Camp", "Leavenworth Federal Penitentiary", "McNeil Island Penitentiary"]
genders = []

data = {'Name': [], 'Gender': [], 'Generation': [], 'Non-Japanese': [], 'BornYear': [], 'Age': [], 'Category': [], 'Born Place': [], 'Camp Location': []}

def calculate_age(birth_year):
    if birth_year is None:
        return None
    current_year = datetime.now().year
    return current_year - int(birth_year)

def extract_locations(text):
    doc = nlp(text)
    born_location = [ent.text for ent in doc.ents if ent.label_ == "GPE"]

    return born_location[-2:]

def extract_camp_locations(text):
    #For storing unique camp names
    found_locations = set()
    for location in camp_locations:
        if location.lower() in text.lower():
            found_locations.add(location)
    return found_locations

#Change file name
zip_file_name = 'DiscoverNikkei.zip'

with zipfile.ZipFile(zip_file_name, 'r') as zip_file:
    for csv_file_name in zip_file.namelist():
        if csv_file_name.endswith('.csv'):
            with zip_file.open(csv_file_name) as csv_file:
                df = pd.read_csv(csv_file, encoding='latin1')

                if 'Name' in df.columns:
                    name = df['Name'].iloc[0]
                    names.append(name)
                    print(name)

                if 'Description' in df.columns:
                    description = df['Description'].iloc[0]

                #Change the column name
                transcripts = df['Transcript (Narrator Only)'].dropna()

                born_year = None
                age = None
                generation_found = None
                nonjap_found = None
                gender_found = None
                individual_camp_locations_found = set()
                bornplace_found = None

                if pd.notna(description):
                    description_sentences = description.split('.')
                    #print('description_sentences', description_sentences)
                    for sentence in description_sentences:
                        if any(word in sentence for word in ['taken', 'center', 'interned ', 'incarceration ', 'removed', 'Removed', 'incarcerated', 'Incarcerated']):
                            locations_found = extract_camp_locations(sentence)
                            if locations_found:
                                individual_camp_locations_found.update(locations_found)
                                print(f"Camp locations mentioned in Description: {', '.join(locations_found)}")
                #If info isn't in descriptions, then find in treanscripts
                if not individual_camp_locations_found:
                    for sentence in transcripts:
                        if any(word in sentence for word in ['Assembly ', 'Assembly Center' 'concentration camp', 'camp', 'removed', 'Removed', 'incarcerated', 'Incarcerated', 'transferred', 'Transferred']):
                            locations_found = extract_camp_locations(sentence)
                            if locations_found:
                                individual_camp_locations_found.update(locations_found)
                                print(f"Camp locations mentioned in Transcript: {', '.join(locations_found)}")

                data['Camp Location'].append(', '.join(individual_camp_locations_found) if individual_camp_locations_found else np.nan)

                if pd.notna(description):
                    description_sentences = description.split('.')
                    #print('description_sentences', description_sentences)
                    for sentence in description_sentences:
                        if sentence.startswith('born') or ('Born'):
                            bornplace_found = extract_locations(sentence)
                            if bornplace_found:
                                print(f"Bornplace Found: {bornplace_found}")
                                break

                if not bornplace_found:
                    for item in transcripts:
                        transcripts_sentences = sentence.split('.')
                        #print('description_sentences', description_sentences)
                        for sentence in transcripts_sentences:
                            if sentence.startswith('born') or ('Born'):
                                match = re.search(r'(\d{4})(?<!20\d{2})', sentence)

                                if match:
                                    born_year = match.group(1)
                                    birthyears.append(born_year)
                                    print("Born Year from Description:", born_year)
                                bornplace_found = extract_locations(sentence)

                                if bornplace_found:
                                    print(f"Bornplace Found: {bornplace_found}")
                                    break

                if pd.notna(description):
                    description_sentences = description.split('.')
                    for sentence in description_sentences:
                        #print("Sentence",sentence)
                        for generation in generations:
                            if generation in sentence.lower():
                                #print(f"Generation mentioned: {generation}")
                                generation_found = generation

                                if generation == 'issei':
                                    issei.append(generation)
                                elif generation == 'nisei':
                                    nisei.append(generation)
                                elif generation == 'sansei':
                                    sansei.append(generation)
                                elif generation == 'yonsei':
                                    yonsei.append(generation)
                                elif generation == 'kibei':
                                    kibei.append(generation)
                                elif generation == 'ni-ten-gosei':
                                    ni_ten_gosei.append(generation)
                                elif generation == 'nisei-sansei':
                                    nisei_sansei.append(generation)

                        for nationality in ['white',  'Suquamish and Filipino descent', 'Jewish']:
                            if re.search(rf'\b{nationality}\b', sentence, re.IGNORECASE):
                                nonjap_found = nationality
                                #print(nonjap_found)

                        #For Densho
                        #for gender in ['male', 'female']:
                            #if re.search(rf'\b{gender}\b', sentence, re.IGNORECASE):
                                #gender_found = gender
                                #print(gender_found)

                        #For JASC
                        if 'her' in sentence.lower():
                            gender = 'female'
                            gender_found = gender
                            print(gender_found)
                        elif 'his' in sentence.lower():
                            gender = 'male'
                            gender_found = gender
                            print(gender_found)

                        if sentence.strip().lower().startswith('born'):
                            match = re.search(r'(\d{4})(?<!20\d{2})', sentence)
                            if match:
                                born_year = match.group(1)
                                birthyears.append(born_year)
                                print("Born Year from Description:", born_year)
                                #break

                if generation_found is None:
                    nogeneration.append(name)
                    print('*** No info about Generation ***')

                if born_year is None:
                    nobirthyear.append(name)
                    #print("*** No info about birthyear ***")

                if born_year is not None:
                    age = calculate_age(born_year)
                    #print(f"Age: {age}")
                #else:
                    #print("Age: NA")

                data['Name'].append(name)
                data['Gender'].append(gender_found)
                data['Generation'].append(generation_found)
                data['Non-Japanese'].append(nonjap_found)
                data['BornYear'].append(born_year)
                data['Born Place'].append(bornplace_found)

                #if born_year is not None:
                    #bornplace_found = extract_locations(sentence)
                    #data['Born Place'].append(bornplace_found)
                #else:
                    #data['Born Place'].append(None)
                #data['Born Place'].append(bornplace_found)

                age = calculate_age(born_year)
                data['Age'].append(age)

                if born_year is not None and age is not None:
                    year = int(born_year)

                    if year > 1942:
                        data['Category'].append('Not born in 1942')
                    elif 1926 <= year <= 1942:
                        data['Category'].append('Minor in 1942')
                    else:
                        data['Category'].append('Adult in 1942')
                else:
                    data['Category'].append(None)

                print('*********************************************')

result_df = pd.DataFrame({key: pd.Series(value) for key, value in data.items()})
result_df.to_csv('new_descriptive_info_DiscoverNikkei.csv', index=False, na_rep='NaN')


A. Wallace Tashima
*********************************************
Aiko Yoshinaga Herzig
Camp locations mentioned in Transcript: Manzanar, Jerome
Born Year from Description: 1924
Born Year from Description: 1924
Born Year from Description: 1924
Born Year from Description: 1924
female
*********************************************
Ben Sakoguchi
Born Year from Description: 1938
Born Year from Description: 1938
Born Year from Description: 1938
Born Year from Description: 1938
Born Year from Description: 1938
Born Year from Description: 1938
Born Year from Description: 1938
Born Year from Description: 1938
*********************************************
Bill Hosokawa
Camp locations mentioned in Transcript: Heart Mountain
Born Year from Description: 1915
*********************************************
Cherry Kinoshita
*********************************************
Chiye Tomihiro
*********************************************
Dale Minami
Camp locations mentioned in Transcript: Santa Anita
Born Year f

In [ ]:
result_df

,Name,Gender,Generation,Non-Japanese,BornYear,Age,Category,Born Place,Camp Location
0,Ed Miyakawa,None,None,None,None,NaN,None,"[Eugene, Oregon]",NaN
1,Tadashi Tsufura,male,None,None,None,NaN,None,"[Seabrook, New Jersey]",Gila River
2,Andre Libante,male,None,None,None,NaN,None,[Los Gatos],San Francisco
3,Shizuko Yamauchi,female,None,white,None,NaN,None,[Poston],NaN
4,Molly Kitajima,None,None,None,None,NaN,None,"[Canada, the west coast]",Tanforan
...,...,...,...,...,...,...,...,...,...
63,Katsumi Hikido,male,None,None,None,NaN,None,[Arizona],"Turlock, Gila River"
64,Paul & Mary Katsuro,male,None,None,None,NaN,None,"[Japan, Peru]",Crystal City
65,Nori masuda,None,None,None,None,NaN,None,[],"Jerome, Fresno, Rohwer"
66,George Iwamoto,male,None,None,None,NaN,None,"[Los Angeles, Manzanar]",NaN


In [ ]:
import pandas as pd

# List of file names
files = ['Densho1_Descriptive_Info_List.csv', 'Densho2_Descriptive_Info_List.csv', 'Densho3_Descriptive_Info_List.csv', 'Densho4_Descriptive_Info_List.csv',
         'JASC_Descriptive_Info_List.csv', 'JAMSJ_Descriptive_Info_List.csv', 'DiscoveNikkei_Descriptive_Info_List.csv']

# Read each CSV file into a DataFrame
dfs = [pd.read_csv(file) for file in files]

# Concatenate DataFrames along rows
combined_df = pd.concat(dfs, ignore_index=True)

# Write the combined DataFrame to a new CSV file
combined_df.to_csv('All_Descriptive_Info_List.csv', index=False, na_rep='NaN')
